# API call from NSRDB for multiple location for multiple year and save locally

### Create a folder where you would like to save the weather data

In [1]:
import requests

url = "https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?api_key=D3m04m2UNPQTXU81Yjq9aBfhAhw7IXQRdOjaIygu"

headers = {
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
}

for year in range(2010, 2019):
    for location in ["37.335480+-121.893028","37.266949+-122.025146","37.386051+-122.083855","37.2350+-121.9570","37.3172+-122.0385"]:
        payload = "names=" + str(year) + "&reason=Academic&mailing_list=true&interval=30&utc=false&email=ritanjali.jena@sjsu.edu&leap_day=false&n=NREL&wkt=POINT(" + location + ")"
        response = requests.request("POST", url, data=payload, headers=headers)
        
        #create a folder in local and give that path here
        file_name = "/Users/ritu/Downloads/apidemoraw/weather_" + str(year) + "_" + location + ".csv"
        
        f = open(file_name, "a")
        f.write(response.text)
        f.close()

# Combine all file into single file

In [2]:
import os
import glob
import pandas as pd

#Give your file path here
os.chdir("/Users/ritu/Downloads/apidemoraw")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv, rename your file name
combined_csv.to_csv( "allraw_combined.csv", index=False, encoding='utf-8-sig')

# Data Pre-Process Code: Combine Hour and Minute, Dropping columns

In [3]:
import numpy as np
def read_data(data_paths):
    """Read the data files and combine them into a single dataset.
    Args:
        data_paths (list): Strings of the paths to the data files.
    Returns:
        A single NumPy array consisting of the data from all datasets
    """
    # Read the first data file
    first_path = data_paths[0]
    full_data = np.genfromtxt(first_path, delimiter=',', skip_header=2,
            names=True)

    # Append all remaining data files
    num_files = len(data_paths)
    for i in range(1, num_files):
        path = data_paths[i]
        new_data = np.genfromtxt(path, delimiter=',', skip_header=2,
                names=True)
        full_data = np.hstack((full_data, new_data))

    return full_data

def trim_vars(data):
    """Trim extraneous variables and observations from the data.
    Args:
        data (ndarray): The full unprocessed dataset
    Returns:
        The full dataset with extraneous variables and observations
        trimmed off.
    """
    # Remove all columns with names in `rm_vars`
    rm_vars = ['Unnamed: 0','DHI', 'DNI', 'Clearsky_DHI', 'Clearsky_DNI',
            'Clearsky_GHI', 'Fill_Flag','Global_Horizontal_UV_Irradiance_280400nm','Global_Horizontal_UV_Irradiance_295385nm', 'f0', 'f1',
       'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12',
       'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20']
    var_names = list(data.dtype.names)
    var_names
    keep_vars = [_ for _ in var_names if _ not in rm_vars]
    #print("keep_vars")
    #print(keep_vars)
    data_trimmed = data[keep_vars]
    return data_trimmed

def recode_time(data):
    """Recode the 'Hour' column to encapsulate both the hour and the
    minute, and then remove the 'Minute' column.
    Args:
        data (ndarray): The full unprocessed dataset
    Returns:
        The full dataset with extraneous variables and observations
    """
    # Recode 'Hour'
    data['Hour'] = data['Hour'] + data['Minute'] / 60

    # Remove 'Minute'
    var_names = list(data.dtype.names)
    keep_vars = [_ for _ in var_names if _ != 'Minute']
    
    data_trimmed = data[:, keep_vars]
    return data_trimmed

def read_and_preprocess():
    """
    """
    data_paths = ['/Users/ritu/Downloads/cupertino_test/cupertino_test.csv']
    dat = read_data(data_paths)
    dat = trim_vars(dat)
    #print("printing")
    #print(dat)

    return dat

if __name__ == "__main__":
    pass

In [4]:
import numpy as np
read_and_preprocess()

array([(2020.,  1.,  1.,  0.,  0., 9. , 7.,  9. , 0., 100.  , 165.51, 0.13, 1023., 1.1, 282., 1.4, 117546.),
       (2020.,  1.,  1.,  0., 30., 9.3, 7.,  9.3, 0., 100.  , 165.16, 0.13, 1023., 1.1, 278., 1.3, 117546.),
       (2020.,  1.,  1.,  1.,  0., 9.6, 7.,  9.6, 0., 100.  , 162.29, 0.13, 1023., 1.2, 274., 1.1, 117546.),
       ...,
       (2019., 12., 31., 22., 30., 4. , 0., -0.1, 0.,  74.4 , 153.97, 0.13, 1021., 0.3,  27., 2.2, 117546.),
       (2019., 12., 31., 23.,  0., 3.7, 0., -0.7, 0.,  72.72, 159.03, 0.13, 1021., 0.3,  29., 2.1, 117546.),
       (2019., 12., 31., 23., 30., 3.4, 0., -0.7, 0.,  74.27, 163.15, 0.13, 1021., 0.3,  29., 2. , 117546.)],
      dtype={'names': ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'Cloud_Type', 'Dew_Point', 'GHI', 'Relative_Humidity', 'Solar_Zenith_Angle', 'Surface_Albedo', 'Pressure', 'Precipitable_Water', 'Wind_Direction', 'Wind_Speed', 'Location'], 'formats': ['<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8

In [5]:
#code to save cleaned and processed combine datafile in local system
import pandas as pd 

#Give the location path here where you wiould like to save the file
pd.DataFrame(read_and_preprocess()).to_csv("//Users/ritu/Downloads/apidemoraw/all_cities_transformed_combined.csv")